# Selenium

Load up the Selenium package. You can find documentation for the package [here](http://selenium-python.readthedocs.io/).

In [ ]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

We'll use Beautiful Soup for parsing.

In [ ]:
from bs4 import BeautifulSoup

And other packages too.

In [ ]:
import os, os.path

## Launching a Browser

In [ ]:
driver_paths = {
    'firefox': os.path.expanduser('~/.local/bin/geckodriver'),
    'chrome': os.path.expanduser('~/.local/bin/chromedriver'),
    'phantom': os.path.expanduser('~/.local/bin/phantomjs'),
}

Only launch **one** of the browsers below!

### Start Chrome

In [ ]:
browser = webdriver.Chrome(executable_path=driver_paths['chrome'])

### Start Firefox

In [ ]:
browser = webdriver.Firefox(executable_path=driver_paths['firefox'])

### Start PhantomJS

[PhantomJS](http://phantomjs.org/) is a headless browser. It can make your scraping very efficient and also works well over a SSH connection.

Since it's pretty obvious that a human isn't driving this browser it makes sense to set a User Agent string.

In [ ]:
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# Make a copy.
capabilities = DesiredCapabilities.PHANTOMJS.copy()
# Add a new User Agent string.
capabilities["phantomjs.page.settings.userAgent"] = (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)"
    "Chrome/61.0.3163.39 Safari/537.36"
)

In [ ]:
browser = webdriver.PhantomJS(executable_path=driver_paths['phantom'], desired_capabilities = capabilities)

In [ ]:
DesiredCapabilities.FIREFOX

### Start Headless Chrome

Recent builds of Chrome are able to launch in a headless state. An overview of getting this working with Selenium can be found [here](https://intoli.com/blog/running-selenium-with-headless-chrome/).

## Configure Selenium

How long should we wait before throwing an exception if an element cannot be found. This is handy because it allows for slow loading pages.

Need only be called once per session.

In [ ]:
browser.implicitly_wait(30)

### Set Window Size & Location

These appear to work best immediately after the browser has been instantiated.

In [ ]:
browser.set_window_size(1000, 800)

In [ ]:
browser.set_window_position(0, 0)

You can set both size and position at once with the `set_window_rect()` method. Another option is to `maximize_window()`.

There are `get_window_size()`, `get_window_position()` and `get_window_rect()`.

## Navigating to Pages

### Navigate to Site

Navigate to [Google](http://www.google.co.za).

In [ ]:
browser.get("http://www.google.com")

Check that we are where we expected to be.

In [ ]:
assert "Google" in browser.title

### Page Properties

Retrieve the URL (not precisely what we asked for).

In [ ]:
browser.current_url

The HTML for the page.

In [ ]:
browser.page_source

### Moving Through Time

You can navigate the browser history using the `forward()` and `backward()` methods.

You can also `refresh()`.

## Navigating within Pages

### Locating Content

A web driver object has a number of methods for locating items on the page:

- `find_element_by_css_selector()`
- `find_element_by_class_name()`
- `find_element_by_id()`
- `find_element_by_name()` - matches the `name` attribute
- `find_element_by_link_text()` and `find_element_by_partial_link_text()`
- `find_element_by_tag_name()`
- `find_element_by_xpath()`.

These methods return a `WebElement` object.

There are corresponding plural versions as well, for example, `find_elements_by_css_selector()`.

These actions can also be chained. So, for example, you could use `find_element_by_id()` to find a particular tag and then use `find_element_by_name()` to search within the context of that tag. This allows you to easily navigate nested tags.

Let's locate the search query text box and the **Google Search** button.

In [ ]:
box_text = browser.find_element_by_id('lst-ib')
box_text

### Element Text

Text content can be retrieved with the `text` attribute.

### Element Attributes

We can extract various characteristics of a web element.

The distinction between *attributes* and *properties* is a little subtle:

- an *attribute* is static while
- a *property* is dynamic.

The `get_attribute()` method will first look for properties then attributes; `get_property()` will only consider properties.

In [ ]:
box_text.get_attribute("maxlength")

In [ ]:
box_text.get_property("value")

Now type something into the search box and evaluate this code chunk again.

We can also check whether a web element is visible and active.

In [ ]:
box_text.is_displayed()

In [ ]:
box_text.is_enabled()

The `is_selected()` method can be used to check whether a check box or radio button is selected.

### Acting on Elements

These are the various actions which can be applied to the current focused element:

- `send_keys()`
- `click()`
- `clear()`.

Let's enter a search term. First go and empty the text box.

In [ ]:
box_text.send_keys("web scraping with python")

Suppose we change our minds. Selenium has a selection of predefined [special keys](http://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.keys).

In [ ]:
# This will delete individual characters.
#
box_text.send_keys(Keys.BACKSPACE)

In [ ]:
# This will clear the element completely.
#
box_text.clear()

Let's enter a more specific search term.

In [ ]:
box_text.send_keys("web scraping with selenium")

To initiate the search we need to press the <kbd>Return</kbd> key.

In [ ]:
box_text.send_keys(Keys.RETURN)

### Action Chains

Action Chains can be used to perform low level interactions with a web page.

Each method on a `ActionChains` object adds interactions to a queue. The events are executed by the `perform()` method.

These are the available actions:

- `click()`
- `click_and_hold()`
- `context_click()`
- `double_click()`
- `drag_and_drop()`
- `drag_and_drop_by_offset()`
- `key_down()`
- `key_up()`
- `move_by_offset()`
- `move_to_element()`
- `release()`
- `send_keys()`
- `send_keys_to_element()`.

Let's select the *Settings* option in the Google menu.

In [ ]:
menu_settings = browser.find_element_by_id('abar_button_opt')
#
assert menu_settings.text == 'Settings'

action = ActionChains(browser)

# Queueing actions one at a time.
#
action.move_to_element(menu_settings)
action.click()
action.perform()

Now let's select the *Search settings* option in the drop down menu.

In [ ]:
# To find the selector I searched for 'Search settings' in the page source.
#
menu_search_settings = browser.find_element_by_css_selector('div._dMq:nth-child(1) > a:nth-child(1)')
#
assert menu_search_settings.text == 'Search settings'

In [ ]:
# Chaining actions.
#
ActionChains(browser).move_to_element(menu_search_settings).click().perform()

### Retrieve Page Contents

You can use all of the machinery of Beautiful Soup to parse the structure of the page. The page contents is accessible via the `page_source` property.

In [ ]:
browser.get('https://www.google.co.za/search?q=web+scraping+with+selenium')

soup = BeautifulSoup(browser.page_source, 'html.parser')

In [ ]:
soup.title

Extract the links for first page of results.

In [ ]:
soup.find_all('cite', attrs = {'class': '_Rm'})

### Executing Scripts

We can execute JavaScript.

In [ ]:
# Scroll to the bottom of the page.
#
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

JavaScript can also be executed asynchronously with `execute_async_script()`.

Use `set_script_timeout()` to determine how long an asynchronous script can run before generating an error.

### Taking Screenshots

There are a few ways to take a screenshot:
    
- `get_screenshot_as_base64()`
- `get_screenshot_as_file()` - saves to a PNG file
- `get_screenshot_as_png()` - returns binary data.

This is not a *full page* screenshot. It only captures the visible area of the page.

In [ ]:
browser.get_screenshot_as_file('google-screenshot.png')

## Browser Stuff

### Waiting for Load

A `WebDriverWait` object can be used to wait for elements on a page to load. It has two methods:

- `until()` - wait for condition to return `True` and
- `until_not()` - wait for condition to return `False`.

In [ ]:
# Wait for #searchform ID to be on page.
#
# timeout = 100
# poll_frequency = 0.25 (default 0.5)
#
WebDriverWait(browser, 100, 0.25).until(lambda b: b.find_element_by_id('searchform'))

If the condition is not satisfied within the expected time then a `TimeoutException` is thrown.

More information about Waits (both explicit and implicit) can be found [here](http://selenium-python.readthedocs.io/waits.html).

### Cookies

Retrieve all cookies for current site.

In [ ]:
browser.get_cookies()

Look at a specific cookie by name.

In [ ]:
browser.get_cookie('NID')

You can add a cookie with `add_cookie()`, which accepts a dictionary.

Cookies can be deleted with `delete_cookie()` and `delete_all_cookies()`.

## Clean Up

In [ ]:
for log in ['geckodriver.log', 'ghostdriver.log', 'google-screenshot.png']:
    try:
        os.remove(log)
    except FileNotFoundError:
        pass

In [ ]:
browser.close()